In [39]:
# Need some packages to talk to Kafka.
# if we have no jars file it will automatic download
# import os
# os.environ['PYSPARK_SUBMIT_ARGS'] = '--packages org.apache.spark:spark-streaming-kafka-0-10_2.12:3.1.2,org.apache.spark:spark-sql-kafka-0-10_2.12:3.1.2 pyspark-shell'

# or set straight with spark configuration
# spark = SparkSession.builder.appName('Streaming')\
#     .config('spark.jars.packages', 'org.apache.spark:spark-streaming-kafka-0-10_2.12:3.1.2,org.apache.spark:spark-sql-kafka-0-10_2.12:3.1.2')\
#     .getOrCreate()

In [40]:
from pyspark.sql import SparkSession
# create spark
spark = SparkSession.builder.appName('Streaming')\
    .config('spark.jars.packages', 'org.apache.spark:spark-streaming-kafka-0-10_2.12:3.1.2,org.apache.spark:spark-sql-kafka-0-10_2.12:3.1.2')\
    .getOrCreate()

In [41]:
# read stream from kafka
df = spark \
      .readStream \
      .format("kafka") \
      .option("kafka.bootstrap.servers", "localhost:9092") \
      .option("subscribe", "DemoTopic") \
      .option("startingOffsets", "earliest") \
      .load()

# .option("failOnDataLoss", "false")\
# df_uber = df.select(col("key").cast("string"),from_json(col("value").cast("string"), schema).alias("value")).select("value.*")
# df_uber.printSchema()
# df.selectExpr("CAST(key AS STRING)", "CAST(value AS STRING)") # convert key-value from binary to string type
df.printSchema()

root
 |-- key: binary (nullable = true)
 |-- value: binary (nullable = true)
 |-- topic: string (nullable = true)
 |-- partition: integer (nullable = true)
 |-- offset: long (nullable = true)
 |-- timestamp: timestamp (nullable = true)
 |-- timestampType: integer (nullable = true)



In [42]:
from pyspark.sql.functions import col, to_timestamp

# Parsing the messeage value into dataframe
df_uber = df.select(col("value").cast("string")).alias("csv").select("csv.*")
df_uber2 = df_uber.selectExpr("split(value,',')[0] as dt",
                               "split(value,',')[1] as lat",
                               "split(value,',')[2] as lon",
                               "split(value,',')[3] as base")
df_uber2.printSchema()

root
 |-- dt: string (nullable = true)
 |-- lat: string (nullable = true)
 |-- lon: string (nullable = true)
 |-- base: string (nullable = true)



In [43]:
# Show df
# df_uber2.writeStream.format("console").outputMode("append").start()

In [ ]:
import time

fomart_datetime = time.strftime('%Y-%m-%d_%H-%M-%S')
path = "hdfs://localhost:9000/raws/raw_{}".format(fomart_datetime)
checkpoint = "hdfs://localhost:9000/checkpoints/checkpoint-{}/".format(fomart_datetime)

In [44]:
# write stream dataframe to hdfs
df_uber2.writeStream.format("csv") \
                    .option("path", path) \
                    .option("checkpointLocation", checkpoint) \
                    .outputMode("append") \
                    .start()

In [45]:
# spark.stop() 
# if using spark.stop() spark structure streaming will genarate "Error in attempt 1 getting Kafka offsets:" and does not write file to sink
# if we using stop must be add .awaitterminate() to write stream